# LSTM model to a Quality Array. Improvements Of Compression Rate

##

In [150]:
from Bio import SeqIO
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
#import matplotlib.pyplot as plt

# NN NN Architectrue
from keras.models import Model#, Sequential
from keras.layers import Input, LSTM, Flatten, Dense, Permute, Dropout
from keras.optimizers import Adam
from keras.regularizers import L1


In [151]:
# Pre-Processing Functions
def seq2bool(seq, amines):
    return (np.expand_dims(np.array(list(seq)),1) == np.expand_dims(amines,1).T)

def qual2bool(qual):
    uq = np.unique(qual)
    return (np.expand_dims(qual,1) == np.expand_dims(uq,1).T)

def qual2R(qual):
    qual = np.array(qual)/(40.0)
    m,s = np.mean(qual), np.std(qual)
    qual = (qual - m)/s
    return qual, m, s;

def R2qual(qual,m,s):
    qual = qual *s + m
    qual = qual*40.0;
    return qual;

#  test
q1 = np.array([1,2,3,0])
q2, m, s = qual2R(q1)
q3 = R2qual(q2,m,s);
sum((q1 - q3)**2)

0.0

In [152]:
# Input- Output Construction Functions
def mk_xt(sequence_array,qual_array, N = 10**2, method = 's'):
    if method == 'q':
        # Quality
        x1 = sequence_array[:N,:];
        x1 = np.moveaxis(np.expand_dims(x1,2), 1,2)
        
        x2 = qual_array[:N,:];
        x2 = np.moveaxis(np.expand_dims(x2,2), 1,2)        

        x = np.concatenate([x1,x2], axis = 2)
        t = np.roll(x2,1)

    elif method == 's':    
        # Sequence
        x = np.moveaxis(np.expand_dims(sequence_array[:N,:],2), 1,2)
        
        t = np.roll(x,1)

    elif method == 'q+s':
        # Sequence + Quality
        x1 = sequence_array[:N,:];
        x1 = np.moveaxis(np.expand_dims(x1,2), 1,2)
        
        x2 = qual_array[:N,:];
        x2 = np.moveaxis(np.expand_dims(x2,2), 1,2)        
        
        x = np.concatenate([x1,x2], axis = 2)
        t = np.roll(x2,1)
    return x,t

def xt2vec(x,t,timesteps,regressors):
    # Vectorizations (one-hot encoding likewise)
    X = [];
    for i in range(timesteps):
        X.append(np.roll(x,-i))

    X = np.concatenate(X, axis = 1)
    t = t[:,0:1,0:regressors];

    t = np.moveaxis(t, 2, 1);
    return X, t;

def mk_arh(units, timesteps, predictors, regressors,learning_rate):
    # Make Architecture
    i = Input(shape=(timesteps,predictors),name = 'Input')
    l = LSTM (units, return_sequences = True, use_bias = False,name = 'LSTM') (i)
    l *= (1.0)/np.tanh(1).astype(float);
    l = Flatten(name = 'Flatten')(l)
    #l = Dropout(0.75,name = 'Dropout')(l)
    l = Dense(regressors, name = 'Dense', activation = 'linear', use_bias = False) (l)

    regressor = Model (inputs = i, outputs = l)
    regressor.compile(optimizer = Adam(lr=learning_rate), loss = 'mean_squared_error', metrics=['mse'])
    return regressor;

def n_params(nn):
    n = 0;
    w = nn.get_weights()
    for ww in w:
        n += ww.size
    return n;

In [153]:
!head -n 12 ../data/E_4_20_1_short_example.fq

'head' is not recognized as an internal or external command,
operable program or batch file.


In [154]:
qual_array, sequence_array = [],[]
nucleotids = ['A', 'C', 'G', 'T']
with open("../data/E_4_20_1_short_example.fq") as input_handle:
    for read in SeqIO.parse(input_handle, "fastq"):
        sequence_array.append(seq2bool(read.seq, nucleotids))
        qual_array.extend(read._per_letter_annotations["phred_quality"])

# Concatenate sequence
sequence_array = 1*np.concatenate(sequence_array, axis = 0)
qual_array,m,s = qual2R(np.array([qual_array]).T)

In [155]:
N = 2**8
method = 'q'; #'s', 'q+s'
n_timesteps = 256;
n_units = 2;
n_regressors = 1;
epochs = 1*10**3;
learning_rate = 0.075;

x,t = mk_xt(sequence_array,qual_array, N, method); #
X,t = xt2vec(x,t,n_timesteps,n_regressors);
n_predictors = X.shape[2];

nn = mk_arh(n_units, n_timesteps,n_predictors, n_regressors, learning_rate)
nn.fit(X, t, epochs = epochs, verbose = 0) # approximate
n = n_params(nn)

C:\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [156]:
y = nn(X)
y = np.expand_dims(y, axis = 2);

y = R2qual(y,m,s);
t = R2qual(t,m,s);
e = y - t

In [157]:
# Error Decay
regressor_df = pd.DataFrame.from_dict(
    {
        'loss': nn.history.history['loss'],
    }
)

error_df = pd.DataFrame.from_dict(
    {
        'error': np.array(e[:,0,0])
    }
)

fig = px.line(regressor_df, y="loss")
fig.show()

fig = px.area(error_df, y='error')
fig.show()

In [158]:
# Summary
nn.summary()
print(nn.input_shape)
print(nn.output_shape)
print(X.shape)
print(t.shape)
print(n_params(nn))

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 5)]          0         
                                                                 
 LSTM (LSTM)                 (None, 256, 2)            56        
                                                                 
 tf.math.multiply_6 (TFOpLam  (None, 256, 2)           0         
 bda)                                                            
                                                                 
 Flatten (Flatten)           (None, 512)               0         
                                                                 
 Dense (Dense)               (None, 1)                 512       
                                                                 
Total params: 568
Trainable params: 568
Non-trainable params: 0
_____________________________________________________________

In [159]:
# Error terms
print((e**2).round().mean())
M = (e**2).round().sum() + n;
print('Compression rate')
print( float(N) / float(M))

0.0
Compression rate
0.4507042253521127


In [160]:
N

256

In [161]:
M

568.0

In [162]:
nn.get_weights()

[array([[ -7.506157  ,  -0.45449555,  -2.3729386 ,   1.7535876 ,
           0.42986226,  -1.275092  ,   4.2221575 ,  -1.7219332 ],
        [ -2.4862049 ,  -4.536675  ,  -1.7778605 ,  -1.5583106 ,
           0.3096445 ,   0.6441257 ,   1.8813475 ,   3.4492323 ],
        [ -4.29595   ,  -6.817821  ,  -2.9804175 ,  -9.560928  ,
          -0.56448865,   0.23020631,   1.3709865 ,   3.7445977 ],
        [  1.5340524 ,   2.8684254 ,   1.7017622 ,  -3.1282768 ,
          -0.31737974,   0.13995187, -10.060885  ,  -4.4295015 ],
        [ -1.699201  ,   3.171605  ,  -0.10369857,   1.4599333 ,
           0.08289372,  -0.26319167,  -4.3028107 ,  -7.504037  ]],
       dtype=float32),
 array([[-5.820959  , -4.95844   ,  2.1910863 , -0.63091445, -1.2599466 ,
          0.68988746, -2.4841444 , -3.8575037 ],
        [ 5.5081353 , -0.51740915,  0.7384066 ,  6.2643337 , -1.4516734 ,
         -3.9141736 , 13.421749  ,  3.9804025 ]], dtype=float32),
 array([[ 7.50675201e-02],
        [ 3.74442041e-01],
    